# FSuSiE Generation
Due to the fact that our results are generated using different version of susiF, I intend to rerun the whole thing using a new and stable version, which is susiF.alpha_0.1.210

## mQTL data
genotype: ROSMAP joint call genotype after MAC 5 and other xqtl pipeline filter, partitioned by cortex TAD list

`/mnt/vast/hpc/csg/molecular_phenotype_calling/eqtl/output/dlpfc_tad_region_list_plink_files/ROSMAP_NIA_WGS.leftnorm.filtered.filtered.plink_files_list.txt`

covariates, generated by yuqi :

`/mnt/vast/hpc/csg/ROSMAP_methy_QTL_beta_minify/data_preprocessing/methyl_QTL_2/covariates/cov_pc_factor.gz`

phenotype,  Post processed by yuqi, based on Yiyi Ma's data: 

`/mnt/vast/hpc/csg/ROSMAP_methy_QTL_beta_minify/data_preprocessing/methyl_QTL_2/phenotype_data/minify_methy_BMIQ.bed.gz`

## haQTL data
genotype: ROSMAP joint call genotype after MAC 5 and other xqtl pipeline filter, partitioned by cortex TAD list

`/mnt/vast/hpc/csg/molecular_phenotype_calling/eqtl/output/dlpfc_tad_region_list_plink_files/ROSMAP_NIA_WGS.leftnorm.filtered.filtered.plink_files_list.txt`

covariates, generated by Xuanhe https://github.com/cumc/fungen-xqtl-analysis/blob/main/analysis/Wang_Columbia/ROSMAP/haqtl/covariates_preprocessing.ipynb :

`/mnt/vast/hpc/csg/ROSMAP_methy_QTL_beta_minify/data_preprocessing/methyl_QTL_2/covariates/cov_pc_factor.gz`

phenotype,  generated by Xuanhe, https://github.com/cumc/fungen-xqtl-analysis/blob/main/analysis/Wang_Columbia/ROSMAP/haqtl/phenotype_preprocessing.ipynb: 

`/mnt/vast/hpc/csg/ROSMAP_methy_QTL_beta_minify/data_preprocessing/methyl_QTL_2/phenotype_data/minify_methy_BMIQ.bed.gz`

Following are the input files

```
hs3163@node89:/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping$ cat uniPhenotype_list
tissue  path
rosmap_haqtl    h3k9ac_whole.bed.gz
yuqi_mqtl       minify_methy_BMIQ.bed.gz
hs3163@node89:/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping$ cat uniCovar_list
tissue  path
rosmap_haqtl    /mnt/mfs/hgrcgrid/homes/xc2610/k9_data/cov_new/h3k9ac_whole.k9_cov.xqtl_protocol_data.filtered.related.filtered.extracted.pca.projected.resid.PEER.merged.gz
yuqi_mqtl       /mnt/vast/hpc/csg/ROSMAP_methy_QTL_beta/data_preprocessing/methyl_QTL_2/covariates/ROSMAP_arrayMethylation_covariates.sesame.methyl.beta.translated.bed_BMIQ.bed.ROSMAP_xqtl_covariates_sex_death_pmi_study.pca.resid.PEER.cov.gz
```

## Running command

```
#!/bin/sh
#$ -l h_rt=220:00:00
#$ -l h_vmem=80G
#$ -N fsusie_tad_rerun
#$ -o /mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/rerun_susie_$JOB_ID.out
#$ -e /mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/rerun_susie_$JOB_ID.err
#$ -j y
#$ -S /bin/bash

export PATH=$HOME/miniconda3/bin:$PATH
source ~/.bashrc

module load Singularity

cd /mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/


sos run pipeline/SuSiE.ipynb uni_fsusie \
    --genoFile /mnt/vast/hpc/csg/molecular_phenotype_calling/eqtl/output/dlpfc_tad_region_list_plink_files/ROSMAP_NIA_WGS.leftnorm.filtered.filtered.plink_files_list.txt  \
    --cwd output/rerun/ --name fsusie \
    --region-list dlpfc_tad_list \
    --phenoFile uniPhenotype_list \
    --covFile uniCovar_list --prior mixture_normal_per_scale  --container containers/stephenslab.sif \
    --max_L 20 \
    --max_SNP_EM 100 \
    -J 20 --job_size 1 -c csg.yml -q csg -s build  --mem 160G --walltime 72h
```

In [5]:
sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/rerun/haQTL --rds_path `ls output/rerun/cache/*haqtl*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif


sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv \
    --cwd output/rerun/mQTL --rds_path `ls output/rerun/cache/*mqtl*rds ` \
    --region-list ../eqtl/dlpfc_tad_list \
    --container containers/stephenslab.sif



SyntaxError: invalid syntax (<ipython-input-5-55f22cd3df90>, line 1)

In [ ]:
ls -lah  output/rerun/cache/*rds  | grep -v K |grep -v M | grep -v G | cut -f 2,3,4 -d "/"  > empty_rds

In [ ]:

nohup sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv --cwd output/rerun/haQTL --rds_path `ls output/rerun/cache/*haqtl*rds | grep -v -f empty_rds ` --region-list ../eqtl/dlpfc_tad_lis
t --container containers/stephenslab.sif -s build &
nohup sos run pipeline/SuSiE_post_processing.ipynb fsusie_to_tsv --cwd output/rerun/mQTL --rds_path `ls output/rerun/cache/*mqtl*rds | grep -v -f empty_rds ` --region-list ../eqtl/dlpfc_tad_lis
t --container containers/stephenslab.sif -s build &

The output are  `/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mappingoutput/rerun/mQTL/fsusie.all_variants.tsv` and `/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mappingoutput/rerun/haQTL/fsusie.all_variants.tsv`